In [ ]:
import torch
import torchvision.models as models
from torchvision.models import resnet50, ResNet50_Weights
import torchvision
from torchvision import transforms
import numpy as np
import onnxruntime as ort
import os
from PIL import Image
import time
import json
import datetime


import onnxruntime
print("Path del modulo importato:", onnxruntime.__file__)
print("Execution Providers disponibili:", onnxruntime.get_available_providers())

In [48]:
def preprocess_image(image_path):
    """Preprocess an image for ResNet-50 ONNX inference."""
    try:
        image = Image.open(image_path).convert('RGB')
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
        image_tensor = preprocess(image).unsqueeze(0).numpy()
        return image_tensor.astype(np.float32)
    except Exception as e:
        raise RuntimeError(f"Error while preprocessing '{image_path}': {e}")

In [49]:
def export_resnet50_onnx(model_path="models/resnet50_imagenet.onnx"):
    if not os.path.exists(model_path):
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        
        model = models.resnet50(pretrained=True).eval()
        
        dummy_input = torch.randn(1, 3, 224, 224)
        torch.onnx.export(
            model, dummy_input, model_path,
            export_params=True, opset_version=13,
            do_constant_folding=True,
            input_names=['input'], output_names=['output'],
            dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
        )
        print("Exported ResNet-50 model to ONNX:", model_path)
    else:
        print("ONNX model already exists:", model_path)

In [50]:
def create_session_onnx(model_path = "models/resnet50_imagenet.onnx"):
    sess_options = ort.SessionOptions()

    providers = ['VitisAIExecutionProvider']
    
    try:
        session = ort.InferenceSession(
            model_path, 
            providers=providers,
            sess_options=sess_options
        )
        
        if 'VitisAIExecutionProvider' not in active_providers:
            print("ATTENZIONE: Il VitisAIExecutionProvider non è stato utilizzato come execution provider.")
            print("Verifica la configurazione del sistema Vitis AI.")
        
    except Exception as e:
        print(f"ERRORE nella creazione della sessione ONNX: {e}")

    return session

In [54]:
session = create_session_onnx()

In [55]:
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
"""
Qui utilizziamo il labels.txt (nella forma --> ILSVRC2012_val_00000001_n01751748.JPEG 65 sea_snake)
per estrarre il path delle immagini nel dataset
"""
def load_imagenet_labels(labels_file="dataset/labels.txt"):
    with open(labels_file) as f:
        labels = [line.strip() for line in f.readlines()]
    return labels

try:
    labels = load_imagenet_labels()
    print(f"Caricate {len(labels)} etichette ImageNet.")
except Exception as e:
    print(f"Errore nel caricamento delle etichette: {e}")

Caricate 50000 etichette ImageNet.


In [56]:
# Funzione di inferenza
def run_inference(image_path, session):
    # Preprocessing dell'immagine
    input_data = preprocess_image(image_path)
    
    # Esecuzione dell'inferenza
    start_time = time.time()
    predictions = session.run([output_name], {input_name: input_data})[0]
    inference_time = time.time() - start_time
    
    # Elaborazione dei risultati
    predictions = predictions[0]  # Rimuovi la dimensione del batch
    top5_indices = np.argsort(predictions)[-5:][::-1]
    top5_results = [(labels[idx], predictions[idx], idx) for idx in top5_indices]
    
    return top5_results, inference_time

print("Funzione di inferenza configurata.")

Funzione di inferenza configurata.


In [57]:
# Parametri MLPerf
print("\nConfigurazione dei parametri MLPerf...")
MLPERF_CONFIG = {
    "framework": "ONNX Runtime",
    "backend": "Vitis AI",
    "hardware": "AMD NPU",
    "model": "ResNet50 v1.5",
    "scenario": "Offline",
    "division": "Open",
    "category": "Image Classification",
    "precision": "FP32",  # Questo cambierà quando quantizzeremo il modello
}

print("Parametri MLPerf configurati:")
for k, v in MLPERF_CONFIG.items():
    print(f"  {k}: {v}")

# Test su un'immagine singola per verifica
test_image = "dataset/val_images_flat/ILSVRC2012_val_00000001_n01751748.JPEG"  # Sostituisci con un'immagine di test

if os.path.exists(test_image):
    print(f"\nEsecuzione di un test di inferenza su {test_image}...")
    top5, inference_time = run_inference(test_image, session)
    
    print(f"Tempo di inferenza: {inference_time*1000:.2f} ms")
    print("Top 5 predizioni:")
    for label, score, idx in top5:
        print(f"  {label}: {score:.2f}% (indice: {idx})")
else:
    print(f"Immagine di test {test_image} non trovata.")


Configurazione dei parametri MLPerf...
Parametri MLPerf configurati:
  framework: ONNX Runtime
  backend: Vitis AI
  hardware: AMD NPU
  model: ResNet50 v1.5
  scenario: Offline
  division: Open
  category: Image Classification
  precision: FP32

Esecuzione di un test di inferenza su dataset/val_images_flat/ILSVRC2012_val_00000001_n01751748.JPEG...
Tempo di inferenza: 26.11 ms
Top 5 predizioni:
  ILSVRC2012_val_00000540_n03825788.JPEG 680 nipple: 55.12% (indice: 539)
  ILSVRC2012_val_00000483_n02814860.JPEG 437 beacon: 52.29% (indice: 482)
  ILSVRC2012_val_00000440_n02097298.JPEG 199 Scotch_terrier: 52.02% (indice: 439)
  ILSVRC2012_val_00000077_n02087394.JPEG 159 Rhodesian_ridgeback: 51.34% (indice: 76)
  ILSVRC2012_val_00000443_n07718747.JPEG 944 artichoke: 50.01% (indice: 442)


In [ ]:
# Funzione per eseguire il benchmark MLPerf
def run_mlperf_benchmark(image_dir="dataset/val_images_flat", batch_process=False, gt_path = "dataset/labels.txt"):
    image_files = []
    with open(gt_path, 'r') as f:
        for line in f:
            filename = line.strip().split()[0]
            filename = image_dir + "/" + filename
            image_files.append(filename)
    
    print(f"\nEsecuzione del benchmark su {len(image_files)} immagini...")

    results = {
        "correct_top1": 0,
        "correct_top5": 0,
        "total": len(image_files),
        "inference_times": [],
        "predictions": []
    }

    start_time_total = time.time()

    for i, img_path in enumerate(tqdm.tqdm(image_files)):
        try:
            top5, inference_time = run_inference(img_path, session)
            results["inference_times"].append(inference_time)

            # Salva le top-5 predizioni come tupla di 5 classi predette
            top5_classes = tuple(pred[2] for pred in top5[:5])
            results["predictions"].append(top5_classes)

        except Exception as e:
            print(f"Errore nell'elaborazione dell'immagine {img_path}: {e}")

    total_time = time.time() - start_time_total

    # Calcola le metriche
    results["total_time"] = total_time
    results["images_per_sec"] = total_images / total_time
    results["avg_inference_time"] = np.mean(results["inference_times"]) if results["inference_times"] else 0
    results["std_inference_time"] = np.std(results["inference_times"]) if results["inference_times"] else 0

    return results

In [ ]:
# Esegui il benchmark completo
print("\nEsecuzione del benchmark MLPerf completo...")

dataset_dir = "dataset/val_images_flat"
full_results = run_mlperf_benchmark(dataset_dir)

# Risultati del benchmark
print("\nRisultati del benchmark MLPerf:")
print(f"  Immagini elaborate: {full_results['total']}")
print(f"  Tempo totale: {full_results['total_time']:.2f} secondi")
print(f"  Throughput: {full_results['images_per_sec']:.2f} immagini/secondo")
print(f"  Latenza media: {full_results['avg_inference_time']*1000:.2f} ms")

# Salvataggio dei risultati nel formato MLPerf
mlperf_results = {
    "mlperf_benchmark": {
        "framework": MLPERF_CONFIG["framework"],
        "backend": MLPERF_CONFIG["backend"],
        "hardware": MLPERF_CONFIG["hardware"],
        "model": MLPERF_CONFIG["model"],
        "scenario": MLPERF_CONFIG["scenario"],
        "division": MLPERF_CONFIG["division"],
        "category": MLPERF_CONFIG["category"],
        "precision": MLPERF_CONFIG["precision"],
        "timestamp": datetime.datetime.now().isoformat(),
    },
    "performance_metrics": {
        "total_images": full_results["total"],
        "total_time_seconds": full_results["total_time"],
        "images_per_second": full_results["images_per_sec"],
        "average_latency_ms": full_results["avg_inference_time"] * 1000,
        "latency_std_ms": full_results["std_inference_time"] * 1000,
    }
}

# Salva i risultati in un file JSON
results_file = "mlperf_results_resnet50_amd_npu.json"
with open(results_file, 'w') as f:
    json.dump(mlperf_results, f, indent=2)

print(f"\nRisultati MLPerf salvati in {results_file}")

In [ ]:
def extract_GT(imagenet_val_map = "dataset/labels.txt"):
    ground_truth = {}
    try:
        with open(imagenet_val_map, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 2:
                    # Il formato è: nome_file ID_classe [nome_classe]
                    image_name = parts[0]
                    class_id = int(parts[1])
                    ground_truth[image_name] = class_id
        
        print(f"Caricate {len(ground_truth)} etichette di ground truth.")
    except Exception as e:
        print(f"Errore nel caricamento del file delle etichette: {e}")
        return None

    return ground_truth

def compute_accuracy_top_1_5(predictions, ground_truth):
    correct_top1 = 0
    correct_top5 = 0
    processed_images = 0

    # Ottieni le etichette GT nello stesso ordine del file
    gt_labels = list(ground_truth.values())

    if len(predictions) != len(gt_labels):
        print(f"Errore: numero di predizioni ({len(predictions)}) diverso dal numero di etichette GT ({len(gt_labels)}).")
        return None

    for pred, gt in zip(predictions, gt_labels):
        processed_images += 1

        if gt == pred[0]:
            correct_top1 += 1
        if gt in pred:
            correct_top5 += 1

    top1_accuracy = correct_top1 / processed_images
    top5_accuracy = correct_top5 / processed_images

    accuracy = {
        "top1": top1_accuracy,
        "top5": top5_accuracy,
        "total_images": processed_images
    }

    print(f"Immagini elaborate: {processed_images}")
    print(f"Accuratezza Top-1: {top1_accuracy*100:.2f}%")
    print(f"Accuratezza Top-5: {top5_accuracy*100:.2f}%")
    
    return accuracy

ground_truth = extract_GT(imagenet_val_map = "dataset/labels.txt")
accuracy_results = compute_accuracy_top_1_5(full_results["predictions"], ground_truth)

# Aggiorna i risultati MLPerf con i dati di accuratezza
mlperf_results["accuracy_metrics"] = {
    "top1_accuracy": accuracy_results["top1"],
    "top5_accuracy": accuracy_results["top5"]
}

# Aggiorna il file dei risultati
with open(results_file, 'w') as f:
    json.dump(mlperf_results, f, indent=2)

print(f"\nRisultati MLPerf aggiornati con le metriche di accuratezza in {results_file}")

In [ ]:
# Generazione del rapporto di submission MLPerf
def generate_mlperf_report(results, config, output_file="mlperf_report.md"):
    """
    Genera un rapporto in formato Markdown per la submission MLPerf
    """
    report = f"""# MLPerf Inference Benchmark Report

## Submission Details

- **Framework**: {config["framework"]}
- **Backend**: {config["backend"]}
- **Hardware**: {config["hardware"]}
- **Model**: {config["model"]}
- **Scenario**: {config["scenario"]}
- **Division**: {config["division"]}
- **Category**: {config["category"]}
- **Precision**: {config["precision"]}
- **Date**: {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

## Performance Metrics

- **Total Images**: {results["performance_metrics"]["total_images"]}
- **Total Time**: {results["performance_metrics"]["total_time_seconds"]:.2f} seconds
- **Throughput**: {results["performance_metrics"]["images_per_second"]:.2f} images/second
- **Average Latency**: {results["performance_metrics"]["average_latency_ms"]:.2f} ms
- **Latency Standard Deviation**: {results["performance_metrics"]["latency_std_ms"]:.2f} ms

## Accuracy Metrics

- **Top-1 Accuracy**: {results["accuracy_metrics"]["top1_accuracy"]*100:.2f}%
- **Top-5 Accuracy**: {results["accuracy_metrics"]["top5_accuracy"]*100:.2f}%

## System Configuration

- **CPU**: AMD Ryzen (specifiche dettagliate)
- **NPU**: AMD NPU (specifiche dettagliate)
- **Memory**: (specifiche del sistema)
- **OS**: (sistema operativo in uso)
- **Software Stack**: Ryzen AI 1.4.0, ONNX Runtime, Vitis AI

## Methodology

This submission follows the MLPerf Inference Rules for the Open Division, Image Classification task.
The benchmark was performed using the ImageNet validation dataset with 50,000 images across 1,000 classes.

### Pre-processing Steps
- Resize to 256x256
- Center crop to 224x224
- Normalize with ImageNet mean and std values

### Post-processing Steps
- Softmax applied to model outputs
- Top-5 classes extracted by confidence

## Notes

This is an experimental submission for academic purposes.
"""
    
    with open(output_file, 'w') as f:
        f.write(report)
    
    return output_file

# Genera il rapporto
report_file = generate_mlperf_report(mlperf_results, MLPERF_CONFIG)
print(f"\nRapporto MLPerf generato in {report_file}")
print("Questo rapporto può essere utilizzato come base per la tua submission MLPerf.")

print("\nIl processo di benchmark per MLPerf è stato completato con successo!")